In [2]:
import openslide
from tqdm import tqdm
from PIL import Image
import numpy as np
import os

def resize_ndpi_image(ndpi_path, output_path, target_size=(2048, 2048)):
    slide = openslide.OpenSlide(ndpi_path)
    
    # Choose a level with reasonable resolution
    level = slide.get_best_level_for_downsample(32)
    thumbnail = slide.read_region((0, 0), level, slide.level_dimensions[level])
    
    # Convert to RGB and resize
    thumbnail = thumbnail.convert("RGB")
    resized = thumbnail.resize(target_size, Image.Resampling.LANCZOS)

    
    resized.save(output_path)

In [6]:
# input_dir = "/Volumes/Maxtor/CONTINUACION Basurto y Cruces"  # 1
# input_dir = "/Volumes/Maxtor/ONKO SCANS TODOS" # 1.5
input_dir = "/Volumes/TOSHIBA EXT/Melanomas IHC" # 2
# input_dir = "/Volumes/EXTERNAL_USB/elkartek-CRUCES"  # 3

# output_dir = "/Users/cameronpreasmyer/Desktop/Images/Basurto and Cruces" # 1
output_dir = "/Users/cameronpreasmyer/Desktop/Images/Melanomas IHC" # 2 
# output_dir = "/Users/cameronpreasmyer/Desktop/Images/Elkartek" # 3

for filename in os.listdir(input_dir):
    print(filename)
    if filename == 'EL-O001-PIR.ndpi':
        pass
    elif filename == 'EL-O001P-ANX - 2018-05-23 22.50.37.ndpi':
        pass
    elif filename.endswith(".ndpi"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace(".ndpi", ".png"))
        resize_ndpi_image(input_path, output_path)

AST-B40240-BCL3 - 2018-07-12 14.38.50.ndpi
AST_B09150-ANX - 2018-05-25 10.16.19.ndpi
AST_B09150-BCL2 - 2018-05-25 10.06.41.ndpi
AST_B09150-BCL3 - 2018-05-25 03.49.56.ndpi
AST_B09150-HE - 2018-05-25 03.22.09.ndpi
AST_B09150-MITF - 2018-05-25 03.58.55.ndpi
AST_B09150-PBP - 2018-05-25 03.40.29.ndpi
AST_B09150-PIR - 2018-05-25 03.31.44.ndpi
AST_B11979-ANX - 2018-05-25 11.17.40.ndpi
AST_B11979-BCL2 - 2018-05-25 11.09.48.ndpi
AST_B11979-BCL3 - 2018-05-25 10.54.01.ndpi
AST_B11979-HE - 2018-05-25 10.25.21.ndpi
AST_B11979-MITF - 2018-05-25 11.02.05.ndpi
AST_B11979-PBP - 2018-05-25 10.46.18.ndpi
AST_B11979-PIR - 2018-05-25 10.37.55.ndpi
AST_B16814-ANX - 2018-05-25 11.53.50.ndpi
AST_B16814-BCL2 - 2018-05-25 11.49.22.ndpi
AST_B16814-BCL3 - 2018-05-25 11.40.13.ndpi
AST_B16814-HE - 2018-05-25 11.25.33.ndpi
AST_B16814-MITF - 2018-05-25 11.44.11.ndpi
AST_B16814-PBP - 2018-05-25 11.35.23.ndpi
AST_B16814-PIR - 2018-05-25 11.30.34.ndpi
AST_B20294-ANX - 2018-05-25 12.50.02.ndpi
AST_B20294-BCL2 - 2018-05-2

In [ ]:
import os
import shutil

keywords = ["PIR", "PBP", "MITF", "HE", "ANX", "BRAF", "BCL2", "BCL3"]
source_folder = "Images/2048Tiles"
#source_folder = "/Users/cameronpreasmyer/Desktop/Images/Melanomas IHC"
#source_folder = "/Users/cameronpreasmyer/Desktop/Images/Elkartek"
destination_root = "Images/"


for filename in os.listdir(source_folder):
    upper_name = filename.upper()
    for keyword in keywords:
        if keyword in upper_name:
            keyword_folder = os.path.join(destination_root, keyword)
            os.makedirs(keyword_folder, exist_ok=True)
            src_path = os.path.join(source_folder, filename)
            dst_path = os.path.join(keyword_folder, filename)
            shutil.copy(src_path, dst_path)
            print(f"Moved {filename} to {keyword_folder}")
            break  # Avoid moving the same file to multiple folders



Moved EL-B216-HE_slide_49.png to Images/HE
Moved EL-O035P-MITF_slide_56.png to Images/MITF
Moved EL-C248-ANX_slide_53.png to Images/ANX
Moved EL-B186-PBP_slide_50.png to Images/PBP
Moved EL-B021ND-PBP_slide_22.png to Images/PBP
Moved EL-C040-BCL3 - 2018-07-12 14.55.02_slide_43.png to Images/BCL3
Moved EL-B013-HE_slide_13.png to Images/HE
Moved EL-O028M-ANX - 2018-05-25 03.10.08_slide_50.png to Images/ANX
Moved EL-B155ND-BCL2_slide_25.png to Images/BCL2
Moved EL-B209NID-HE_slide_39.png to Images/HE
Moved EL-C240 Mx-BCL2_slide_30.png to Images/BCL2
Moved EL-C048-BCL3_slide_43.png to Images/BCL3
Moved EL-B209NID-PBP_slide_17.png to Images/PBP
Moved EL-C037N-BCL3bis_slide_26.png to Images/BCL3
Moved EL-B212M1A-MITF_slide_41.png to Images/MITF
Moved AST_B16814-HE - 2018-05-25 11.25.33_slide_23.png to Images/HE
Moved EL-C011-PIR_slide_59.png to Images/PIR
Moved EL-C240 Mx-BCL2_slide_12.png to Images/BCL2
Moved EL-C238-ANX_slide_36.png to Images/ANX
Moved EL-C048-BCL3_slide_36.png to Images/B

In [ ]:
def extract_tiles(slide_path, output_dir, tile_size=(512, 512), stride=512):
    slide = openslide.OpenSlide(slide_path)
    width, height = slide.dimensions
    os.makedirs(output_dir, exist_ok=True)

    # Calculate total number of tiles
    total_tiles = ((height - tile_size[1]) // stride + 1) * ((width - tile_size[0]) // stride + 1)
    count = 0

    with tqdm(total=total_tiles, desc="Extracting tiles") as pbar:
        for y in range(0, height, stride):
            for x in range(0, width, stride):
                if x + tile_size[0] <= width and y + tile_size[1] <= height:
                    tile = slide.read_region((x, y), 0, tile_size).convert("RGB")

                    # Replace the below with whats in the comment beneath it if filtering for informative tiles
                    tile.save(os.path.join(output_dir, f"tile_{count}.png"))
                    '''
                    if is_informative(tile):
                        tile.save(os.path.join(output_dir, f"tile_{count}.png"))
                    '''
                    count += 1
                    pbar.update(1)

In [ ]:
input_ndpi = "/Users/cameronpreasmyer/Desktop/Inputtest/EL-C021-ANX.ndpi"
output_tiles = "/Users/cameronpreasmyer/Desktop/outputtest/tiles"

extract_tiles(input_ndpi, output_tiles, tile_size=(1024, 1024), stride=1024)